In [ ]:
!touch /tmp/tmp.txt
!touch /tmp/tmp0.txt
!touch /tmp/tmp1.txt
!touch /tmp/tmp2.txt
!touch /tmp/tmp3.txt
!touch /tmp/tmp4.txt

In [ ]:
from taigapy import default_tc as tc

In [ ]:
new_dataset_id = tc.create_dataset(
    "CN latest",
    dataset_description="copy number data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "expression",
    dataset_description="expression data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp0.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "mutations latest",
    dataset_description="mutations latest data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp1.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "fusions",
    dataset_description="fusions data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp2.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "CN Achilles version",
    dataset_description="CN Achilles version data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp3.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "dmc_21Q3",
    dataset_description="dmc_21Q3", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="aee7ec053d434091a670bc64a9d7a3c1", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "ibm_21Q3",
    dataset_description="ibm_21Q3", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="aee7ec053d434091a670bc64a9d7a3c1", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "internal_21Q3",
    dataset_description="internal_21Q3", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="aee7ec053d434091a670bc64a9d7a3c1", # optional, will default to your home folder if not provided
)

In [ ]:
new_dataset_id = tc.create_dataset(
    "public_21Q3",
    dataset_description="public_21Q3", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="aee7ec053d434091a670bc64a9d7a3c1", # optional, will default to your home folder if not provided
)

In [ ]:
import numpy as np
from gsheets import Sheets
from depmapomics import tracker as track
from depmapomics import utils
from depmapomics.qc import cn
from depmapomics.config import *
from IPython.display import Image, display
import dalmatian as dm
import pandas as pd
from taigapy import TaigaClient
tc = TaigaClient()
import os
from genepy import mutations as mut
from genepy.utils import helper as h
from genepy import terra
from genepy.google import gcp
from genepy import rna
import matplotlib.pyplot as plt


SHEETCREDS = '../.credentials.json'
MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"
REFSHEET_URL = "https://docs.google.com/spreadsheets/d/161mmmHF5nc3nFhgpp_7zub5Erod8xDbXnNZ-u-7BWKI"
SHEETNAME = 'ccle sample tracker sandbox'

my_id=MY_ID
mystorage_id=MYSTORAGE_ID
sheetcreds=SHEETCREDS
sheetname=SHEETNAME
refsheet_url=REFSHEET_URL

In [ ]:
sheets = Sheets.from_files(my_id, mystorage_id)
tracker = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)



In [ ]:
import dalmatian as dm

wm1 = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WGS_CN-sandbox")

wgs_samples = wm1.get_samples()
wgs_samples = wgs_samples[10:13]
# wgs_samples.to_csv("wgs_samples.tsv", sep = '\t')

In [ ]:
wm2 = dm.WorkspaceManager("broad-firecloud-ccle/simone-test")
# wm2 = wm2.disable_hound()
wm2.upload_samples(wgs_samples)